# Timing CRVI 

In [ ]:
import numpy as np
from scipy.stats import norm, gamma
import matplotlib.pyplot as plt
import cvxpy as cp
import time 

In [ ]:
np.random.seed(42)

# Parameters (# of samples, # of features, beta params, alpha params)
n = 100
p = 5
lambda_ = 2
a0 = 10.0  
b0 = 15.0  

def generate_samples():
    # Simulate X
    X = np.random.randn(n, p)  
    
    # Simulate alpha 
    alpha = gamma.rvs(1, loc = a0, scale = 1 / b0)
    
    # Simulate beta 
    beta = np.random.normal(0, np.sqrt(1 / lambda_), p)
    
    # Simulate y
    y = np.random.normal(X @ beta, 1 / np.sqrt(alpha))

    return (X, y)

In [ ]:
# Define the objective function as trace(FA)
def compute_F_0(X, y):
    A_0 = np.zeros((3 + 2 * p + p ** 2, 3 + 2 * p + p ** 2))

    # Define all of the terms in A_0:
    
    # For mu^T A_block u (sum of outer product of columns of X)
    A_0[3 : 3 + p, 3 + p: 3 + 2 * p] = 0.5 * np.sum([np.outer(X[i], X[i]) for i in range(n)], axis = 0)
    
    # For e A_block Sigma
    A_0[2][3 + 2 * p :] = (0.5 * np.sum([np.outer(X[i], X[i]) for i in range(n)], axis = 0)).ravel()
    
    # for mu^T lambda / 2 mu
    A_0[3 : 3 + p, 3 : 3 + p] = np.eye(p) * lambda_ * 0.5
    
    b_0 = np.zeros(3 + 2 * p + p ** 2)
    
    # Define all the terms in b_0
    b_0[2] = 0.5 * np.sum(np.square(y)) + b0
    b_0[3 + p: 3 + 2 * p] = - np.sum(X, axis = 0) # x_i @ u
    
    # trace term
    b_0[3 + 2 * p :] = (np.eye(p) * lambda_ * 0.5).ravel()
    
    # Now we can define F as we do in the paper
    F_0 = np.zeros((4 + 2 * p + p ** 2, 4 + 2 * p + p ** 2)) 
    F_0[0, 1:] = 0.5 * b_0
    F_0[1:, 0] = 0.5 * b_0
    F_0[1:, 1:] = A_0
    
    return F_0

In [ ]:
# Converting the other constraints into trace(FA) 

# Equality contraint for a:
def compute_F_1(X, y):
    b_1 = np.zeros(3 + 2 * p + p ** 2)
    b_1[0] = 1
    F_1 = np.zeros((4 + 2 * p + p ** 2, 4 + 2 * p + p ** 2)) 
    F_1[0, 1:] = 0.5 * b_1
    F_1[1:, 0] = 0.5 * b_1
    
    return F_1

# Equality constraint for c:
def compute_F_2(X, y):
    b_2 = np.zeros(3 + 2 * p + p ** 2)
    b_2[1] = 1
    F_2 = np.zeros((4 + 2 * p + p ** 2, 4 + 2 * p + p ** 2)) 
    F_2[0, 1:] = 0.5 * b_2
    F_2[1:, 0] = 0.5 * b_2
    return F_2

# Equality constraint for ac - e = 0
def compute_F_3(X, y):
    A_3 = np.zeros((3 + 2 * p + p ** 2, 3 + 2 * p + p ** 2))
    A_3[0, 1] = 1
    b_3 = np.zeros(3 + 2 * p + p ** 2)
    b_3[2] = -1
    F_3 = np.zeros((4 + 2 * p + p ** 2, 4 + 2 * p + p ** 2)) 
    F_3[0, 1:] = 0.5 * b_3
    F_3[1:, 0] = 0.5 * b_3
    F_3[1:, 1:] = A_3
    
    return F_3

# e >= 0
def compute_F_4(X, y):
    b_4 = np.zeros(3 + 2 * p + p ** 2)
    b_4[2] = 1
    F_4 = np.zeros((4 + 2 * p + p ** 2, 4 + 2 * p + p ** 2)) 
    F_4[0, 1:] = 0.5 * b_4
    F_4[1:, 0] = 0.5 * b_4

    return F_4

In [ ]:
# Implement the optimization:

trials = 1000

# Define variables
A = cp.Variable((4 + 2 * p + p ** 2, 4 + 2 * p + p ** 2), symmetric=True)  
a = cp.Variable(nonneg=True)  
c = cp.Variable(nonneg=True)   
Sigma = cp.Variable((p, p), symmetric=True)  

ranks = np.zeros(trials)
times = np.zeros(trials)

for t in range(trials):

    X, y = generate_samples()
    F_0 = compute_F_0(X,y)
    F_1 = compute_F_1(X,y)
    F_2 = compute_F_2(X,y)
    F_3 = compute_F_3(X,y)
    F_4 = compute_F_4(X,y)
    
    # Objective function components for f_1 without the loggamma term:
    term6 = - (n / 2) * (cp.log(a) -(1 / 2) * cp.inv_pos(a) -(1 / 12) * (cp.inv_pos(a) ** 2)  + cp.log(c)) 
    term7 = - (1 / 2) * cp.log_det(Sigma) 
    term8 = - a - cp.log(c) 
    term9 = - cp.log(a) +(1 / 2) * cp.inv_pos(a) +(1 / 12) * (cp.inv_pos(a) ** 2) - cp.entr(a) +(1/2) + (1 / 12) * (cp.inv_pos(a)) 
    term10 = -(a0 - 1) * cp.log(c)
    term11 = -(a0 - 1) * (cp.log(a) -(1 / 2) * cp.inv_pos(a) -(1 / 12) * (cp.inv_pos(a) ** 2))
    f_1 = term6 + term7 + term8 + term9 + term10  + term11 
    
    # Define objective function 
    objective = cp.Minimize(cp.trace(F_0 @ A) + f_1)
    
    # Some basic constrains, A is psd, Sigma is psd, a>0, c > 0, tr(F1A) = a, tr(F2A) = c, ac - e = 0, e >= 0
    constraints = []
    constraints.append(A >> 0)
    constraints.append(A[0][0] == 1)
    constraints.append(Sigma >> 0)
    constraints.append(a >= 0)
    constraints.append(c >= 0)
    constraints.append(cp.trace(F_1 @ A) == a)
    constraints.append(cp.trace(F_2 @ A) == c)
    constraints.append(cp.trace(F_3 @ A) == 0)
    constraints.append(cp.trace(F_4 @ A) >= 0)
    
    # Trace(F_ij A) = sigma_ij
    for i in range(p**2):
        b = np.zeros(3 + 2 * p + p ** 2)
        b[3 + 2 * p + i] = 1
        F = np.zeros((4 + 2 * p + p ** 2, 4 + 2 * p + p ** 2)) 
        F[0, 1:] = 0.5 * b
        F[1:, 0] = 0.5 * b
        row = i // p  
        col = i % p  
    
        constraint = cp.trace(F @ A) == Sigma[row, col]
        constraints.append(constraint)
    
    # e mu_i = u_i constraints:
    for i in range(p):
        A_k = np.zeros((3 + 2 * p + p ** 2, 3 + 2 * p + p ** 2))
        A_k[2, 3 + i] = 1
        b_k = np.zeros(3 + 2 * p + p ** 2)
        b_k[3 + p + i] = -1
        F_k = np.zeros((4 + 2 * p + p ** 2, 4 + 2 * p + p ** 2))
        F_k[0, 1:] = 0.5 * b_k
        F_k[1:, 0] = 0.5 * b_k
        F_k[1:, 1:] = A_k
        constraints.append(cp.trace(F_k @ A) == 0)
        
    # Add trace constraints
    #constraints.append(cp.norm(A, "nuc") <= 3)
    constraints.append(cp.trace(A) <= 10)
    
    # Give the objective and constraints
    problem = cp.Problem(objective, constraints)

    s_time = time.time()
    # SCS is a conic solver and CLARABEL is an interior point solver
    problem.solve(solver=cp.SCS, verbose=False)
    #problem.solve(solver=cp.CLARABEL, verbose=True)
    times[t] = time.time() - s_time
    
    #print("Optimal value:", problem.value)
    # print("Optimal A:", A.value)
    # print("Optimal a:", a.value)
    # print("Optimal c:", c.value)
    # print("Optimal Sigma:", Sigma.value)
    eigenvalues = np.linalg.eigvals(A.value)
    #print("Rank of A:", np.sum(eigenvalues > 1e-4))
    ranks[t] = np.sum(eigenvalues > 1e-4)

In [1]:
print("Average time:", np.mean(times))
print("Average rank:", np.mean(ranks))
#print(ranks)

Average time: 0.37083310294151306
Average rank: 2.851
